In [2]:
pip install librosa soundfile pyaudio

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [5]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            stft = np.abs(librosa.stft(X))
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
        return result

In [6]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [9]:
#DataFlair - Load the data and extract features for each sound file
import os
import glob
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("F://SER//dataset_SER//Actor_*//*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


In [10]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [11]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [12]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [13]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [14]:
#DataFlair - Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [15]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)

In [16]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 52.60%


In [17]:
import sounddevice as sd
import numpy as np
import librosa
from sklearn.neural_network import MLPClassifier


In [18]:
# Function to extract features from audio input
def extract_features_from_audio(audio, sample_rate):
    # Extract features using librosa (e.g., MFCCs)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
    mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
    return np.mean(mfccs.T, axis=0), np.mean(chroma.T, axis=0), np.mean(mel.T, axis=0)

In [19]:
# Function to predict emotion from audio input
def predict_emotion_from_audio(audio, sample_rate, model):
    # Extract features
    mfcc, chroma, mel = extract_features_from_audio(audio, sample_rate)
    features = np.hstack((mfcc, chroma, mel))
    # Predict emotion using the trained model
    emotion = model.predict(features.reshape(1, -1))[0]
    return emotion

In [20]:
# Capture audio input from microphone
def capture_audio(duration=3, sample_rate=22050):
    print("Recording...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float64')
    sd.wait()
    print("Recording stopped.")
    return audio.flatten(), sample_rate

In [21]:
# Real-time emotion prediction
def real_time_emotion_prediction(model):
    while True:
        input("Press Enter to start recording...")
        audio, sample_rate = capture_audio()
        emotion = predict_emotion_from_audio(audio, sample_rate, model)
        print("Predicted emotion:", emotion)

In [22]:
# Run real-time emotion prediction
real_time_emotion_prediction(model)

Recording...
Recording stopped.
Predicted emotion: fearful
Recording...
Recording stopped.
Predicted emotion: fearful
Recording...
Recording stopped.
Predicted emotion: fearful
Recording...
Recording stopped.
Predicted emotion: fearful
Recording...
Recording stopped.
Predicted emotion: disgust
Recording...
Recording stopped.
Predicted emotion: disgust
Recording...


KeyboardInterrupt: 